In [1]:
# Basic imports & versions
import sys, os, json, time, math, pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch as th

# SB3
from stable_baselines3 import SAC
from stable_baselines3.common.monitor import Monitor

# Utils and CityLearn
from utils.env_utils import keep_only_electrical_storage, keep_only_core_observations, PPOTrainLogger
import citylearn
from citylearn.data import DataSet
from citylearn.citylearn import CityLearnEnv
from citylearn.wrappers import NormalizedObservationWrapper, StableBaselines3Wrapper

print('Python:', sys.version.split()[0])
print('Torch :', th.__version__)
print('CityLearn:', citylearn.__version__)

Couldn't import dot_parser, loading of dot files will not be possible.


2025-08-26 00:04:03.033670: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-26 00:04:03.047473: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756159443.063255 3645259 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756159443.067629 3645259 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756159443.079371 3645259 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Python: 3.11.7
Torch : 2.7.0+cu126
CityLearn: 2.4.1


In [2]:
# Configuration (consistent with PPO/A2C)
DATASET_NAME = 'citylearn_challenge_2023_phase_3_1'
ROOT_DIR = r'/hkfs/home/haicore/iai/cj9272/Bachelorthesis_DQN_Agent/data/datasets/citylearn_challenge_2023_phase_3_1'
REWARD_FN = {  # CostReward
    'type': 'citylearn.reward_function.CostReward',
    'attributes': {}
}
PRICING_FILE = 'pricing_germany_2023_june_to_august.csv'
ONLY_ELECTRICAL_STORAGE = True
EVAL_EPISODES = 10

# Action label helpers (for comparability with RBC/DQN plots)
ACTION_LABELS = ['strong_discharge','mild_discharge','idle','mild_charge','strong_charge']
NAME_TO_FRAC = {
    'strong_discharge': -1.0,
    'mild_discharge': -0.5,
    'idle': 0.0,
    'mild_charge': 0.5,
    'strong_charge': 1.0,
}

In [3]:
# --- Load schema ---
dataset = DataSet()
schema = dataset.get_schema(DATASET_NAME)
schema['root_directory'] = ROOT_DIR
schema['reward_function'] = REWARD_FN

# Set pricing file on all buildings
price_file = PRICING_FILE
if 'buildings' not in schema:
    raise RuntimeError("schema does not contain 'buildings'")
for bname, bconf in schema['buildings'].items():
    bconf['pricing'] = price_file

# Keep only electrical storage and core obs (same as other notebooks)
schema = keep_only_electrical_storage(schema)
schema = keep_only_core_observations(schema, extra_keep=['carbon_intensity'], drop_predictions=False)

# Create base env
env = CityLearnEnv(schema, central_agent=True)

# Basic checks
print('Initial time step:', env.time_step)
print('Number of time steps:', env.time_steps)
print('Central agent:', env.central_agent)
print('Number of buildings:', len(env.buildings))
print('Electrical storage capacity:', {b.name: b.electrical_storage.capacity for b in env.buildings})
print('Action_space:', env.action_space)
print('Number of Observations:', len(env.observations[0]))

INFO:root:Go here /home/iai/cj9272/.cache/citylearn/v2.4.1/datasets/citylearn_challenge_2023_phase_3_1/schema.json 


/hkfs/home/haicore/iai/cj9272/Bachelorthesis_DQN_Agent/data/datasets/citylearn_challenge_2023_phase_3_1
Dataset '/hkfs/home/haicore/iai/cj9272/Bachelorthesis_DQN_Agent/data/datasets/citylearn_challenge_2023_phase_3_1' copied to '/hkfs/home/haicore/iai/cj9272/Bachelorthesis_DQN_Agent/data/datasets/citylearn_challenge_2023_phase_3_1/../../../../results/2025-08-26_00-04-06'
Initial time step: 0
Number of time steps: 2208
Central agent: True
Number of buildings: 6
Electrical storage capacity: {'Building_1': 4.0, 'Building_2': 4.0, 'Building_3': 3.3, 'Building_4': 3.3, 'Building_5': 4.0, 'Building_6': 3.3}
Action_space: [Box(-1.0, 1.0, (6,), float32)]
Number of Observations: 20


In [4]:
# Build & Train SAC (continuous actions; no discretization needed)
log_dir = os.path.join(os.getcwd(), 'logs', 'sac')
os.makedirs(log_dir, exist_ok=True)

train_env = NormalizedObservationWrapper(env)
train_env = StableBaselines3Wrapper(train_env)
train_env = Monitor(train_env, filename=os.path.join(log_dir, "monitor.csv"))

logger = PPOTrainLogger()  # generic callback capturing ep/step stats

TOTAL_TIMESTEPS = 2208 * 40

model = SAC(
    policy='MlpPolicy',
    env=train_env,
    seed=0,
    device='cpu',
    verbose=1,
    learning_rate=3e-4,
    buffer_size=200_000,
    batch_size=256,
    tau=0.005,
    gamma=0.99,
    ent_coef='auto',
    train_freq=(1, 'step'),
    gradient_steps=1,
    policy_kwargs=dict(net_arch=[256, 256], activation_fn=th.nn.ReLU),
)

t0 = time.time()
model.learn(total_timesteps=TOTAL_TIMESTEPS, callback=logger)
print(f"Total time elapsed for {TOTAL_TIMESTEPS} steps: {(time.time()-t0):.2f} seconds")
model.save("sac_01")

Using cpu device
Wrapping the env in a DummyVecEnv.
Total time elapsed for 88320 steps: 0.06 seconds


In [5]:
# Eval env (same wrappers)
eval_env = CityLearnEnv(schema, central_agent=True)
eval_env = NormalizedObservationWrapper(eval_env)
eval_env = StableBaselines3Wrapper(eval_env)

# Optionally reload (or use in-memory model)
# model = SAC.load("sac_01")

# Enhanced evaluation: collect actions and rewards, print KPIs and timings.
def evaluate_with_metrics(model, env, episodes=5, deterministic=True, render=False):
    ep_metrics, all_step_rewards, all_kpis, all_actions = [], [], [], []
    t_global_start = time.time()
    for ep in range(1, episodes+1):
        obs, _ = env.reset()
        done = False
        ep_ret, ep_len = 0.0, 0
        step_rewards = []
        action_list = []
        t_ep_start = time.time()
        while not done:
            action, _ = model.predict(obs, deterministic=deterministic)
            action_list.append(np.array(action))
            obs, r, terminated, truncated, info = env.step(action)
            done = bool(terminated or truncated)
            ep_ret += float(r)
            step_rewards.append(float(r))
            ep_len += 1
            if render:
                env.render()
        kpis = env.unwrapped.evaluate()
        all_kpis.append(kpis)
        all_actions.append(np.vstack(action_list))
        t_ep = time.time() - t_ep_start
        steps_per_sec = ep_len / max(t_ep, 1e-9)
        ep_metrics.append({
            'episode': ep, 'cumulative reward': ep_ret, 'length': ep_len,
            'mean_reward': ep_ret/ep_len if ep_len else np.nan,
            'min_step_reward': float(np.min(step_rewards)),
            'max_step_reward': float(np.max(step_rewards)),
            'std_step_reward': float(np.std(step_rewards)),
            'steps_per_sec': steps_per_sec, 'wall_time_s': t_ep
        })
        all_step_rewards.extend(step_rewards)
        print(f"[Eval] Ep {ep}/{episodes} return={ep_ret:.3f} len={ep_len} "
              f"mean={ep_ret/ep_len:.3f} time={t_ep:.2f}s speed={steps_per_sec:.1f} steps/s")
        # print("KPIs:", kpis)
    print(f"Total eval time: {time.time()-t_global_start:.2f}s  | Avg/ep: {(time.time()-t_global_start)/episodes:.2f}s")
    metrics_df = pd.DataFrame(ep_metrics)
    display(metrics_df)
    return metrics_df, np.array(all_step_rewards), all_kpis, all_actions

# Run evaluation
metrics_stoch, step_rewards_stoch, kpis_stoch, actions_stoch = evaluate_with_metrics(
    model, eval_env, episodes=EVAL_EPISODES, deterministic=False
)

/hkfs/home/haicore/iai/cj9272/Bachelorthesis_DQN_Agent/data/datasets/citylearn_challenge_2023_phase_3_1
Dataset '/hkfs/home/haicore/iai/cj9272/Bachelorthesis_DQN_Agent/data/datasets/citylearn_challenge_2023_phase_3_1' copied to '/hkfs/home/haicore/iai/cj9272/Bachelorthesis_DQN_Agent/data/datasets/citylearn_challenge_2023_phase_3_1/../../../../results/2025-08-26_00-04-09'
[Eval] Ep 1/10 return=-1719.148 len=2207 mean=-0.779 time=282.98s speed=7.8 steps/s
[Eval] Ep 2/10 return=-1720.526 len=2207 mean=-0.780 time=271.55s speed=8.1 steps/s
[Eval] Ep 3/10 return=-1731.370 len=2207 mean=-0.784 time=301.77s speed=7.3 steps/s
[Eval] Ep 4/10 return=-1723.673 len=2207 mean=-0.781 time=301.89s speed=7.3 steps/s
[Eval] Ep 5/10 return=-1734.389 len=2207 mean=-0.786 time=301.85s speed=7.3 steps/s
[Eval] Ep 6/10 return=-1724.375 len=2207 mean=-0.781 time=295.89s speed=7.5 steps/s
[Eval] Ep 7/10 return=-1723.442 len=2207 mean=-0.781 time=157.46s speed=14.0 steps/s
[Eval] Ep 8/10 return=-1727.508 len=2

,episode,cumulative reward,length,mean_reward,min_step_reward,max_step_reward,std_step_reward,steps_per_sec,wall_time_s
0,1,-1719.147636,2207,-0.778952,-4.859519,0.0,0.655825,7.799071,282.982430
1,2,-1720.525804,2207,-0.779577,-4.858914,0.0,0.649378,8.127477,271.547986
2,3,-1731.369847,2207,-0.784490,-5.596459,0.0,0.665878,7.313416,301.774145
3,4,-1723.673095,2207,-0.781003,-4.579178,0.0,0.649517,7.310537,301.893019
4,5,-1734.388899,2207,-0.785858,-5.200827,0.0,0.667839,7.311602,301.849016
5,6,-1724.374941,2207,-0.781321,-4.449775,0.0,0.655506,7.458736,295.894647
6,7,-1723.441887,2207,-0.780898,-4.893862,0.0,0.651756,14.016424,157.458134
7,8,-1727.507542,2207,-0.782740,-4.590384,0.0,0.660531,14.012317,157.504283
8,9,-1727.933981,2207,-0.782933,-5.164514,0.0,0.660553,14.072130,156.834818
9,10,-1722.632911,2207,-0.780531,-5.442038,0.0,0.665439,14.027390,157.335042


In [6]:
# Quick learning check
if hasattr(logger, 'ep_df') and len(logger.ep_df):
    ep_df = logger.ep_df
    first_n, last_n = 10, 10
    first_mean = ep_df['return'].head(first_n).mean()
    last_mean  = ep_df['return'].tail(last_n).mean()
    print(f'First {first_n} ep mean return: {first_mean:.3f}')
    print(f'Last  {last_n} ep mean return: {last_mean:.3f}')
    print('Learning indicator (last - first):', f'{(last_mean-first_mean):.3f}')
else:
    print('No episode returns in logger.ep_df')

No episode returns in logger.ep_df


In [1]:
# Plots (same as PPO/A2C; quantize actions to 5 bins for comparability)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Choose a building to visualize
building_name = 'Building_1'
bld_names = [b.name for b in eval_env.unwrapped.buildings]
if building_name not in bld_names:
    building_name = bld_names[0]
b_idx = bld_names.index(building_name)
print(f'Plotting actions for building: {building_name} (index {b_idx})')

# Flatten evaluated actions across episodes -> shape (total_steps, n_actions)
actions_arr = np.concatenate(actions_stoch, axis=0)  # each item shape (ep_len, n_actions)
n_actions = actions_arr.shape[1]
total_steps = actions_arr.shape[0]

# Build episode boundaries from metrics
ep_lengths = metrics_stoch['length'].to_numpy()
ep_offsets = np.cumsum(np.concatenate([[0], ep_lengths[:-1]]))
hour_index = np.arange(total_steps)

# Load pricing for the chosen building
price_file = schema['buildings'][building_name]['pricing']
price_path = os.path.join(ROOT_DIR, price_file)
prc = pd.read_csv(price_path)
T = len(prc)  # usually 2208

# Map continuous action to 5 RBC-like labels by nearest of [-1,-0.5,0,0.5,1]
bin_centers = np.array([-1.0, -0.5, 0.0, 0.5, 1.0])
ACTION_LABELS = ['strong_discharge','mild_discharge','idle','mild_charge','strong_charge']
def cont_to_label(vals):
    idx = np.argmin(np.abs(vals[..., None] - bin_centers[None, ...]), axis=-1)
    return idx  # 0..4

# ----- Plot 1: Actions over Time (for selected building) -----
window = 500
act_series = actions_arr[:, b_idx]
act_ma = pd.Series(act_series).rolling(window, min_periods=1).mean().to_numpy()

plt.figure(figsize=(7,3))
plt.scatter(hour_index, act_series, s=5, alpha=0.35, label='raw')
plt.plot(hour_index, act_ma, color='C2', lw=2, label=f'{window}-step MA')
plt.title(f'Actions over time (raw + MA) - {building_name}')
plt.xlabel('Step'); plt.ylabel('Action [-1..1]'); plt.grid(True); plt.legend(loc='upper right')
plt.tight_layout(); plt.show()

# ----- Plot 2: Action Distribution (quantized to 5 bins like RBC) -----
labels_idx = cont_to_label(act_series)
counts = pd.Series(labels_idx).value_counts().reindex(range(5)).fillna(0).astype(int)
plt.figure(figsize=(4,3))
ax = counts.plot(kind='bar', color=['C0','C1','C2','C3','C4'])
ax.set_xticklabels(ACTION_LABELS, rotation=45, ha='right')
total = counts.sum()
ax.bar_label(ax.containers[0], labels=[f'{int(v)}\n({v/total:.1%})' for v in counts.values])
ax.set_ylim(0, counts.max()*1.5)
plt.ylabel('Count'); plt.title('Action Distribution (5-bin)'); plt.tight_layout(); plt.show()

# ----- Plot 3: Episode 1 Reward MA vs Electricity Price -----
step_rewards_all = step_rewards_stoch  # flattened across eval episodes
ep1_len = int(ep_lengths[0])
ep1_rewards = step_rewards_all[:ep1_len]
window_r = 200
ep1_reward_ma = pd.Series(ep1_rewards).rolling(window_r, min_periods=1).mean().to_numpy()

fig, ax1 = plt.subplots(figsize=(7,3))
ax1.plot(np.arange(ep1_len), ep1_reward_ma, color='C3', label=f'Reward MA (w={window_r})')
ax1.set_ylabel(f'MA Reward ({window_r})', color='C3'); ax1.tick_params(axis='y', labelcolor='C3')
ax1.set_xlabel('Hour (episode 1)'); ax1.grid(True)

ax2 = ax1.twinx()
ax2.plot(prc.index[:ep1_len], prc['electricity_pricing'][:ep1_len], color='C0', alpha=0.5, label='Electricity Price')
ax2.set_ylabel('Electricity Price', color='C0'); ax2.tick_params(axis='y', labelcolor='C0')

lines = ax1.get_lines() + ax2.get_lines()
ax1.legend(lines, [l.get_label() for l in lines], loc='lower left')
plt.title(f'Episode 1: Reward MA vs Electricity Price - {building_name}')
plt.tight_layout(); plt.show()

# ----- Plot 4: Full-horizon Reward vs Electricity Price (tiled across episodes) -----
window_reward = 500
reward_ma = pd.Series(step_rewards_all).rolling(window_reward, min_periods=1).mean().to_numpy()
n_hours_total = len(step_rewards_all)
n_repeats = int(np.ceil(n_hours_total / T))
price_tiled = np.tile(prc['electricity_pricing'].values, n_repeats)[:n_hours_total]
price_ma = pd.Series(price_tiled).rolling(24, min_periods=1).mean().to_numpy()

fig, ax1 = plt.subplots(figsize=(12,4))
ax1.plot(np.arange(n_hours_total), reward_ma, color='C3', label=f'Reward MA (w={window_reward})')
ax1.set_xlabel('Step'); ax1.set_ylabel('Reward (MA)', color='C3'); ax1.tick_params(axis='y', labelcolor='C3'); ax1.grid(True, axis='x', alpha=0.4)

ax2 = ax1.twinx()
ax2.plot(np.arange(n_hours_total), price_ma, color='C0', lw=2, label='Price 24h MA')
ax2.set_ylabel('Electricity Price', color='C0'); ax2.tick_params(axis='y', labelcolor='C0')

# Mark episode boundaries
for off in ep_offsets[1:]:
    ax1.axvline(off, color='k', linestyle='--', alpha=0.15, linewidth=0.8)

lines = ax1.get_lines() + ax2.get_lines()
ax1.legend(lines, [l.get_label() for l in lines], loc='upper right', frameon=False)
plt.title('Eval Reward vs Electricity Pricing (tiled)'); plt.tight_layout(); plt.show()

# ----- Plot 5: Solar Generation vs Reward MA Across Eval Episodes -----
bld_file = schema['buildings'][building_name]['energy_simulation']
bld_path = os.path.join(ROOT_DIR, bld_file)
bld_df = pd.read_csv(bld_path)
solar_vals = bld_df['solar_generation'].values

solar_tiled = np.tile(solar_vals, int(np.ceil(n_hours_total / T)))[:n_hours_total]

fig, ax1 = plt.subplots(figsize=(12,4))
ax1.plot(np.arange(n_hours_total), reward_ma, color='C3', label=f'Reward MA (w={window_reward})')
ax1.set_ylabel('Reward (MA)', color='C3'); ax1.tick_params(axis='y', labelcolor='C3'); ax1.grid(True, axis='x', alpha=0.3)
ax2 = ax1.twinx()
ax2.plot(np.arange(n_hours_total), solar_tiled, color='C2', alpha=0.75, label='Solar Generation')
ax2.set_ylabel('Solar Generation', color='C2'); ax2.tick_params(axis='y', labelcolor='C2')

lines = ax1.get_lines() + ax2.get_lines()
ax1.legend(lines, [l.get_label() for l in lines], loc='upper right', frameon=False)
plt.title(f'Solar Generation vs Reward MA Across {len(ep_lengths)} Eval Episodes - {building_name}')
plt.tight_layout(); plt.show()

NameError: name 'eval_env' is not defined